In [3]:
import librosa
import opensmile
import os
import sys
import numpy as np
import random
from collections import defaultdict
from copy import deepcopy

In [84]:
class LoadKG:
    
    def __init__(self):
        
        self.x = 'Hello'
        
    def load_train_data(self, data_path, one_hop, data, s_t_r, entity2id, id2entity,
                     relation2id, id2relation):
        
        data_ = set()
    
        ####load the train, valid and test set##########
        with open (data_path, 'r') as f:
            
            data_ini = f.readlines()
                        
            for i in range(len(data_ini)):
            
                x = data_ini[i].split()
                
                x_ = tuple(x)
                
                data_.add(x_)
        
        ####relation dict#################
        index = len(relation2id)
                        
        for key in data_:
            
            if key[1] not in relation2id:
                
                relation = key[1]
                
                relation2id[relation] = index
                
                id2relation[index] = relation
                
                index += 1
                
                #the inverse relation
                iv_r = '_inverse_' + relation
                
                relation2id[iv_r] = index
                
                id2relation[index] = iv_r
                
                index += 1
        
        #get the id of the inverse relation, by above definition, initial relation has 
        #always even id, while inverse relation has always odd id.
        def inverse_r(r):
            
            if r % 2 == 0: #initial relation
                
                iv_r = r + 1
            
            else: #inverse relation
                
                iv_r = r - 1
            
            return(iv_r)
        
        ####entity dict###################
        index = len(entity2id)
        
        for key in data_:
            
            source, target = key[0], key[2]
            
            if source not in entity2id:
                                
                entity2id[source] = index
                
                id2entity[index] = source
                
                index += 1
            
            if target not in entity2id:
                
                entity2id[target] = index
                
                id2entity[index] = target
                
                index += 1
                
        #create the set of triples using id instead of string        
        for ele in data_:
            
            s = entity2id[ele[0]]
            
            r = relation2id[ele[1]]
            
            t = entity2id[ele[2]]
            
            if (s,r,t) not in data:
                
                data.add((s,r,t))
            
            s_t_r[(s,t)].add(r)
            
            if s not in one_hop:
                
                one_hop[s] = dict()
            
            if r not in one_hop[s]:
                
                one_hop[s][r] = set()
            
            one_hop[s][r].add(t)
            
            if t not in one_hop:
                
                one_hop[t] = dict()
            
            r_inv = inverse_r(r)
            
            s_t_r[(t,s)].add(r_inv)
            
            if r_inv not in one_hop[t]:
                
                one_hop[t][r_inv] = set()
            
            one_hop[t][r_inv].add(s)

In [85]:
#define the dictionaries and sets for load KG
one_hop = dict() 
data = set()
data_ini = set()
entity2id = dict()
id2entity = dict()
relation2id = dict()
id2relation = dict()
s_t_r = defaultdict(set)

#read the training KG triples
Class_1 = LoadKG()

#fill in the sets and dicts
Class_1.load_train_data('../../KG-TACT/data/WN18RR_v2_ind/train.txt', one_hop, data, s_t_r,
                        entity2id, id2entity, relation2id, id2relation)

Class_1.load_train_data('../../KG-TACT/data/WN18RR_v2_ind/valid.txt', one_hop, data, s_t_r,
                        entity2id, id2entity, relation2id, id2relation)

Class_1.load_train_data('../../KG-TACT/data/WN18RR_v2_ind/test.txt', one_hop, data, s_t_r,
                        entity2id, id2entity, relation2id, id2relation)

In [86]:
print(len(relation2id), len(entity2id), len(data))

20 2757 4863


In [1]:
priority_list = [[(1,2),5], [(3,4),1], [(5,6),3]]

In [2]:
priority_list.sort(key = lambda x: x[-1])

In [3]:
priority_list

[[(3, 4), 1], [(5, 6), 3], [(1, 2), 5]]